異体字のペアを作成する

In [1]:
%cd itaiji

/home/analysis01/src/kanjikana-model/dict/itaiji


/home/analysis01/src/kanjikana-model/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%cd tobunken

/home/analysis01/src/kanjikana-model/dict/itaiji/tobunken


In [3]:
!wget --no-check-certificate -O index.html https://www.tobunken.go.jp/archives/%E7%95%B0%E4%BD%93%E5%AD%97%E3%83%AA%E3%82%B9%E3%83%88/

--2025-09-29 13:43:08--  https://www.tobunken.go.jp/archives/%E7%95%B0%E4%BD%93%E5%AD%97%E3%83%AA%E3%82%B9%E3%83%88/
www.tobunken.go.jp (www.tobunken.go.jp) をDNSに問いあわせています... 202.223.176.4
www.tobunken.go.jp (www.tobunken.go.jp)|202.223.176.4|:443 に接続しています... 接続しました。
警告: `www.tobunken.go.jp' の証明書は信用されません。
警告: `www.tobunken.go.jp' の証明書の発行者が不明です。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 特定できません [text/html]
`index.html' に保存中

index.html              [ <=>                ] 921.03K  --.-KB/s 時間 0.05s      

2025-09-29 13:43:08 (18.6 MB/s) - `index.html' へ保存終了 [943132]



In [4]:
!python extract.py --infile="index.html" --outfile=tobunken_itaiji.txt

In [5]:
%cd ../

/home/analysis01/src/kanjikana-model/dict/itaiji


ペアにして、属性をつける。 

key1,key2:[attr1,attr2]
key2,key1:[attr1,attr2]      

In [6]:
pair={}

In [7]:
def make_pair(fname,attr):
    with open(fname,'r',encoding='utf-8') as f:
        for l in f:
            items=l.rstrip().split(',')
            keyA=f'{items[0]},{items[1]}'      
            keyB=f'{items[1]},{items[0]}'
            if keyA not in pair:
                pair[keyA]=[]
            if keyB not in pair:
                pair[keyB]=[]
            pair[keyA].append(attr)
            alst=pair[keyA]
            pair[keyB].append(attr)
            blst=pair[keyB]
            pair[keyA]=list(set(alst))
            pair[keyB]=list(set(blst))

In [8]:
make_pair('tobunken/tobunken_itaiji.txt','tobunken_itaiji')

In [9]:
len(pair)

52788

In [10]:
make_pair('../oss/mozc/itaiji.txt','mozc_itaiji')

In [11]:
len(pair)

52812

In [12]:
make_pair('../oss/skk/itaiji.txt','skk_itaiji')

In [13]:
len(pair)

53256

In [14]:
pair

{'皑,皚': ['tobunken_itaiji'],
 '皚,皑': ['tobunken_itaiji'],
 '皑,凒': ['tobunken_itaiji'],
 '凒,皑': ['tobunken_itaiji'],
 '皚,凒': ['tobunken_itaiji'],
 '凒,皚': ['tobunken_itaiji'],
 '蔼,譪': ['tobunken_itaiji'],
 '譪,蔼': ['tobunken_itaiji'],
 '蔼,藹': ['tobunken_itaiji'],
 '藹,蔼': ['tobunken_itaiji'],
 '藹,譪': ['tobunken_itaiji'],
 '譪,藹': ['tobunken_itaiji'],
 '霭,謁': ['tobunken_itaiji'],
 '謁,霭': ['tobunken_itaiji'],
 '霭,谒': ['tobunken_itaiji'],
 '谒,霭': ['tobunken_itaiji'],
 '霭,謁': ['tobunken_itaiji'],
 '謁,霭': ['tobunken_itaiji'],
 '霭,䨠': ['tobunken_itaiji'],
 '䨠,霭': ['tobunken_itaiji'],
 '霭,靄': ['tobunken_itaiji'],
 '靄,霭': ['tobunken_itaiji'],
 '靄,謁': ['tobunken_itaiji'],
 '謁,靄': ['tobunken_itaiji'],
 '靄,谒': ['tobunken_itaiji'],
 '谒,靄': ['tobunken_itaiji'],
 '靄,謁': ['tobunken_itaiji'],
 '謁,靄': ['tobunken_itaiji'],
 '靄,䨠': ['tobunken_itaiji'],
 '䨠,靄': ['tobunken_itaiji'],
 '爱,愛': ['tobunken_itaiji'],
 '愛,爱': ['tobunken_itaiji'],
 '爱,㤅': ['tobunken_itaiji'],
 '㤅,爱': ['tobunken_itaiji'],
 '愛,㤅': ['tobu

In [15]:
siz={}
for k,v in pair.items():
    if len(v) not in siz:
        siz[len(v)]=0
    siz[len(v)]+=1

In [16]:
siz

{1: 51366, 3: 1218, 2: 672}

同じ文字のグループを作成する

In [17]:
groups=[]
for k,v in pair.items():
    key1,key2=k.split(',')
    for i in range(len(groups)):
        if key1 in groups[i]:
            groups[i].append(key2)
            groups[i]=list(set(groups[i]))
            break
        elif key2 in groups[i]:
            groups[i].append(key1)
            groups[i]=list(set(groups[i]))
            break
    else:
        groups.append([key1,key2])


In [18]:
groups

[['皑', '凒', '皚'],
 ['譪', '蔼', '藹'],
 ['䨠', '謁', '霭', '謁', '靄', '谒'],
 ['㤅', '愛', '爱'],
 ['暧', '瞹', '曖'],
 ['諳', '谙'],
 ['鏕', '鏖'],
 ['鰲', '鳌', '鼇'],
 ['媪', '媼'],
 ['叐', '㧞', '抜', '拔', '杮'],
 ['罴', '羆', '罷', '罢'],
 ['捭', '䙓', '㪏', '擺', '摆', '襬'],
 ['唄', '㗑', '呗'],
 ['敗', '贁', '败', '䢙'],
 ['朌', '攽', '颁', '䫞', '頒'],
 ['鈑', '版', '钣'],
 ['绊', '靽', '絆'],
 ['帮', '㿶', '縍', '鞤', '幫', '幇', '䩷', '㨍', '幚'],
 ['謗', '谤'],
 ['䳰', '鳵', '鴇', '駂', '䳈', '鸨'],
 ['飽', '饱', '飹', '䭋'],
 ['报', '報'],
 ['蚫', '鲍', '鮑'],
 ['鵯', '鹎'],
 ['貝', '蛽', '贝'],
 ['辈', '軰', '輩'],
 ['狈', '䟺', '狽', '䟛'],
 ['俻', '葡', '備', '偹', '备', '僃'],
 ['坟', '墳', '賁', '辬', '隫', '贲'],
 ['繃', '綳', '䙖', '䙀', '绷'],
 ['筆', '笔'],
 ['縪', '韠', '罼', '彃', '畢', '鏎', '㪤', '毕'],
 ['幣', '幤', '币', '印'],
 ['闭', '閈', '閇', '閉', '闬'],
 ['獙', '弊', '斃', '敝', '獘', '毙'],
 ['蓽', '篳', '荜', '筚'],
 ['腗', '髀', '䃾', '䯗', '䏶', '脾', '肶'],
 ['缏', '緶', '编', '編'],
 ['邉', '辺', '邊', '边'],
 ['貶', '贬', '疺', '乏'],
 ['変', '㣐', '變', '变'],
 ['辫', '辮'],
 ['办', '覍', '弁', '麰', '瓣', '辨

In [19]:
len(groups)

5228

第一水準から順に優先順をつけて、優先のものに代替するようにする
さらに小中高の順に優先する

In [20]:
def load_jis(fname):
    with open(fname,'r',encoding='utf-8') as f:
        hsh ={l.rstrip():1 for l in f if len(l.rstrip())>0}
    return hsh

In [21]:
dai1=load_jis('../daitai/jisdai1.dat')

In [22]:
dai2=load_jis('../daitai/jisdai2.dat')

In [23]:
dai3=load_jis('../daitai/jisdai3.dat')

In [24]:
dai4=load_jis('../daitai/jisdai4.dat')

In [25]:
syo=load_jis('../daitai/syogaku.dat')

In [26]:
cyu=load_jis('../daitai/cyugaku.dat')

In [27]:
kou=load_jis('../daitai/koukou.dat')

In [28]:
suijun=[syo,cyu,kou,dai1,dai2,dai3,dai4]

In [29]:
import regex
import re

In [30]:
def get_suijun(s):
    # 平仮名カタカナは最優先 0でかえす  旧字は除外
    if (regex.match('^\p{Script=Hiragana}+$',s) is not None or regex.match('^\p{Script=Katakana}+$',s) is not None) and re.match(f'^[ゐゑ	゚゛゜ゝゞゟ]',s) is None and re.match('^[ヰヱヸヹヽヾヿ]+$',s) is None:
        return 0 
    
    for i,sui in enumerate(suijun):
        if s in sui:
            return i+1
    return len(suijun) # 1-4水準外

In [31]:
def get_min_suijun(lst): # 配列内で最も水準が高いものの水準
    min_suijun=5
    min_moji=''
    for l in lst:
        sui=get_suijun(l)
        if min_suijun>sui:
            min_suijun=sui
            min_moji=l
    if min_suijun==5:
        #min_moji=lst[0]  # 全部1ー4水準外の時は最初のものを選択
        # 最もコードポイントが小さいもの
        clst=[ord(l) for l in lst]
        min_moji=chr(min(clst))
        print(min_moji)
    return min_suijun,min_moji        

In [32]:
ngroup={}  # キーが水準の低いもののみ残す  {key2:key1,key3:key1,key4:key1}  key1 is min suijun

In [33]:
for vv in groups:
    min_sui,min_moji=get_min_suijun(vv)
    for val in vv:
        if val =="䨠" or min_moji=="䨠":
            continue
        if val == min_moji:
            continue
        key=f'{min_moji},{val}'
        attr=[]
        if key in pair:
            attr=list(set(pair[key]))
        ngroup[val]={min_moji:{"dics":attr}}


凒
蔼
諳
鏕
鰲
媪
㪏
絆
㨍
謗
蚫
鵯
䙀
筚
䃾
辫
猋
鰾
蟞
㯽
繽
傧
殡
髩
拨
驂
惭
凔
厕
恻
㑪
婵
䜛
諂
僤
怅
衬
齓
騁
䲭
炽
㿧
筹
綢
䟞
刍
闖
怆
莼
㝇
綽
罬
枞
揝
篡
䞼
挞
紿
殚
郸
惮
鐺
㨶
䚐
呧
嵮
瘨
魼
鶇
窦
牍
㸿
䫳
㾄
錞
炖
朵
亸
䳗
諤
閼
鶚
珐
矾
魴
鯡
冯
諷
䴸
輻
凫
賻
鰒
溉
秆
䧻
肐
巩
覯
䧸
蛊
詁
䡰
㥤
雚
閨
詭
滚
鮌
埚
䈐
駭
頜
亢
䧚
㗿
闔
黉
汻
扈
奂
涣
焕
鰉
詼
秽
荟
誨
諱
荤
珲
諢
㘉
㞆
譏
䝴
䪠
䍤
緝
葪
荠
覬
蘎
浃
荚
賈
歼
囏
拣
睑
枧
湔
䬻
浆
娇
浇
鷦
㦘
轎
訐
頡
䒺
饉
㶳
縉
覲
賮
刭
㘫
劤
痉
胫
䦰
匮
䶥
剧
寠
颶
鋑
䳌
皲
隽
忾
龕
㯺
軻
㧶
䭝
誑
㚀
篑
餽
㾐
籁
賚
篮
䍀
闌
㦨
㧛
㰖
䌫
痨
䛶
㰚
㰛
沥
珞
茘
轢
礰
匲
敛
脸
㶑
俩
蜽
繆
䨅
璙
㐭
伈
蔺
蹸
餾
珑
茏
垄
蝼
䫫
篓
瘘
鏤
㿖
䡎
魲
轆
峦
挛
圝
閭
埨
綸
萝
鑼
䯁
駱
榈
䮫
縷
妈
玛
迈
瞒
㗄
䊡
鏝
䆀
们
㨉
㦖
覓
䛑
㥸
悯
黽
謨
䮬
呒
㕯
挠
蛲
鐃
閙
鯢
腻
輦
䙚
嗫
蹑
䉲
顳
咛
䡤
狞
聍
泞
㺲
駑
疟
傩
瓯
㰶
蹒
憉
毞
諞
䮁
騗
縹
驃
姘
鮃
㸄
䭶
碛
訖
佥
悭
䮿
䈤
譴
㟻
椠
跄
鏘
樯
蔷
呛
䂪
窍
㚁
誚
匧
㒌
㻽
䶚
郥
硂
顴
綣
闕
㹱
麇
䫇
桡
荛
娆
蝾
縟
䬃
腮
啬
魦
筛
删
澹
殇
觞
㒤
㵕
虱
埘
諡
軾
贖
閂
駟
怂
耸
誦
謖
嚯
虽
㺚
闥
駘
摊
䜖
鐔
㢭
䥖
髫
粜
恸
偷
䫋
飩
釶
駝
鴕
腽
帓
綄
帏
䆧
莴
䠎
呜
誈
㒇
䃖
覡
䦧
娴
痫
藓
䗾
膁
䊑
㕺
䲷
潇
箫
䔥
驍
啸
纈
紲
舋
汹
銹
勖
㾌
鉉
荨
咰
娅
訝
剦
壛
鴦
炀
䖹
鷂
晔
靥
䞅
䨨
檥
䆿
怿
縊
㸧
嘤
珱
䓨
䨉
塋
瑩
贏
㿈
䊸
俞
楡
欤
䛕
伛
屿
齬
妪
閾
鶐
橼
轅
䤦
䵴
紜
䆬
涢
蕰
錾
凿
枣
啧
箦
譖
牐
毡
噡
崭
榐
撵
栈
顫
赵
蛰
嘀
輒
鷓
䡩
䧵
峥
錚
㡠
踯
掷
贄
㭽
鷙
㤘
紂
駎
㦵
銖
伫
啭
賺
騅
䋘


In [34]:
len(ngroup)

11543

In [35]:
ngroup

{'皑': {'凒': {'dics': ['tobunken_itaiji']}},
 '皚': {'凒': {'dics': ['tobunken_itaiji']}},
 '譪': {'蔼': {'dics': ['tobunken_itaiji']}},
 '藹': {'蔼': {'dics': ['tobunken_itaiji']}},
 '霭': {'謁': {'dics': ['tobunken_itaiji']}},
 '謁': {'謁': {'dics': ['tobunken_itaiji']}},
 '靄': {'謁': {'dics': ['tobunken_itaiji']}},
 '谒': {'謁': {'dics': ['tobunken_itaiji']}},
 '㤅': {'愛': {'dics': ['tobunken_itaiji']}},
 '爱': {'愛': {'dics': ['tobunken_itaiji']}},
 '暧': {'曖': {'dics': ['tobunken_itaiji']}},
 '瞹': {'曖': {'dics': ['mozc_itaiji', 'skk_itaiji']}},
 '谙': {'諳': {'dics': ['tobunken_itaiji']}},
 '鏖': {'鏕': {'dics': ['tobunken_itaiji']}},
 '鳌': {'鰲': {'dics': ['tobunken_itaiji']}},
 '鼇': {'鰲': {'dics': ['skk_itaiji', 'mozc_itaiji', 'tobunken_itaiji']}},
 '媼': {'媪': {'dics': ['tobunken_itaiji']}},
 '叐': {'抜': {'dics': ['tobunken_itaiji']}},
 '㧞': {'抜': {'dics': ['tobunken_itaiji']}},
 '拔': {'抜': {'dics': ['skk_itaiji', 'mozc_itaiji', 'tobunken_itaiji']}},
 '杮': {'抜': {'dics': ['tobunken_itaiji']}},
 '罴': {'

In [36]:
import json

In [37]:
with open('../itaiji_o.json','w',encoding='utf-8') as f:
    json.dump(ngroup,f,ensure_ascii=False,indent=2)

In [38]:
!date

2025年  9月 29日 月曜日 13:43:46 JST
